In [1]:
!pip install onnx onnx_tf
!git clone https://github.com/kartikbatra056/CelebFaces-Attributes-Prediction.git

import torch
import tensorflow as tf
import onnx
from torch.autograd import Variable
from onnx_tf.backend import prepare

     |████████████████████████████████| 12.7 MB 8.0 MB/s 
     |████████████████████████████████| 222 kB 45.1 MB/s 
     |████████████████████████████████| 1.1 MB 53.6 MB/s 
Cloning into 'CelebFaces-Attributes-Prediction'...
remote: Enumerating objects: 97, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 97 (delta 23), reused 1 (delta 0), pack-reused 53
Unpacking objects: 100% (97/97), done.


In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
import torchvision.transforms as transforms
from PIL import Image
import io
# Defining Class for Single Layer.
class Layer(nn.Module):
    def __init__(self,in_ch,out_ch,kernel_size=3,stride=1,padding=1):
        super(Layer,self).__init__()
        self.conv=nn.Conv2d(in_ch,out_ch,kernel_size,stride,padding)
        self.bn=nn.BatchNorm2d(out_ch)
        self.relu=nn.ReLU()
        nn.init.xavier_uniform_(self.conv.weight)
    def forward(self,Input):
        output=self.conv(Input)
        output=self.bn(output)
        output=self.relu(output)
        return output

class Permute(nn.Module):
    def __init__(self, *args):
        super(Permute, self).__init__()
        self.order = args

    def forward(self, x):
        return x.permute(self.order)

# Define complete model
class CelebModel(nn.Module):
    def __init__(self,num_classes=40):
        super(CelebModel,self).__init__()

        self.unit0 = Permute(0, 3, 1, 2)
        self.unit1=Layer(in_ch=3,out_ch=32)
        self.unit2=Layer(in_ch=32,out_ch=32)
        self.pool1=nn.MaxPool2d(kernel_size=2)

        self.unit3=Layer(in_ch=32,out_ch=64)
        self.unit4=Layer(in_ch=64,out_ch=64)
        self.pool2=nn.MaxPool2d(kernel_size=2)

        self.unit5=Layer(in_ch=64,out_ch=128)
        self.unit6=Layer(in_ch=128,out_ch=128)
        self.unit7=Layer(in_ch=128,out_ch=128)
        self.pool3=nn.MaxPool2d(kernel_size=2)

        self.unit8=Layer(in_ch=128,out_ch=256,kernel_size=5,padding=0)
        self.unit9=Layer(in_ch=256,out_ch=256,kernel_size=5,padding=0)
        self.unit10=Layer(in_ch=256,out_ch=256,kernel_size=5,padding=0)
        self.pool4=nn.MaxPool2d(kernel_size=2)

        self.drop2=nn.Dropout(0.5)

        self.unit11=Layer(in_ch=256,out_ch=512,kernel_size=3,padding=0)
        self.unit12=Layer(in_ch=512,out_ch=512,kernel_size=3,padding=0)
        self.unit13=Layer(in_ch=512,out_ch=512,kernel_size=3,padding=0)

        self.pool5=nn.AvgPool2d(kernel_size=2)

        self.drop3=nn.Dropout(0.5)

        self.model=nn.Sequential(self.unit1,self.unit2,self.pool1,self.unit3,
                                 self.unit4,self.pool2,self.unit5,self.unit6,
                                 self.unit7,self.pool3,self.unit8,self.unit9,
                                 self.unit10,self.pool4,self.drop2,self.unit11,
                                 self.unit12,self.unit13,self.pool5,self.drop3)

        self.fc=nn.Linear(in_features=512,out_features=num_classes)

    def forward(self,Input):
        
        output = self.unit0(Input)
        output=self.model(output)
        output=output.view(-1,512)
        output=self.fc(output)

        return output

# creating model object
model=CelebModel()

# load weights
model.load_state_dict(torch.load('/content/CelebFaces-Attributes-Prediction/model.pth',map_location=torch.device('cpu')))

# set dropout and batch normalization layers to evaluation mode
model.eval()

CelebModel(
  (unit0): Permute()
  (unit1): Layer(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (unit2): Layer(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (unit3): Layer(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (unit4): Layer(
    (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (pool2): MaxPool2d(kernel_size=2, stride=2, 

In [5]:
def transform_image(image_bytes):
    transform=transforms.Compose([transforms.Resize((224,224)),
                              transforms.ToTensor(),
                              transforms.Normalize(mean=[0.5063, 0.4258, 0.3832],std=[0.2644, 0.2436, 0.2397])])
    image=Image.open(io.BytesIO(image_bytes)).convert('RGB')
    return transform(image).permute(1, 2, 0)

def Predict(image):
    pred=model(image.unsqueeze(0))
    labels=['5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive', 'Bags_Under_Eyes',
       'Bald', 'Bangs', 'Big_Lips', 'Big_Nose', 'Black_Hair', 'Blond_Hair',
       'Blurry', 'Brown_Hair', 'Bushy_Eyebrows', 'Chubby', 'Double_Chin',
       'Eyeglasses', 'Goatee', 'Gray_Hair', 'Heavy_Makeup', 'High_Cheekbones',
       'Male', 'Mouth_Slightly_Open', 'Mustache', 'Narrow_Eyes', 'No_Beard',
       'Oval_Face', 'Pale_Skin', 'Pointy_Nose', 'Receding_Hairline',
       'Rosy_Cheeks', 'Sideburns', 'Smiling', 'Straight_Hair', 'Wavy_Hair',
       'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Lipstick',
       'Wearing_Necklace', 'Wearing_Necktie', 'Young']
    attr=list(torch.round(torch.sigmoid(pred)).cpu().detach().numpy().squeeze(0))
    prd=list(torch.sigmoid(pred).cpu().detach().numpy().squeeze(0))
    new_labels=[label for label,a in list(zip(labels,attr)) if a==1]
    pred_list=[p for p,a in list(zip(prd,attr)) if a==1]
    return [{'name':label ,'percent':round(p,2)} for label,p in zip(new_labels,pred_list)]

# Test image
with open("/content/statham.jpeg", "rb") as file:
    img_bytes=file.read()

tensor=transform_image(img_bytes)
labels=Predict(tensor)

In [7]:
labels

[{'name': 'Big_Nose', 'percent': 0.71},
 {'name': 'Chubby', 'percent': 0.59},
 {'name': 'Male', 'percent': 0.99},
 {'name': 'Mouth_Slightly_Open', 'percent': 0.65},
 {'name': 'Mustache', 'percent': 0.77},
 {'name': 'Wearing_Hat', 'percent': 1.0}]

In [9]:
trained_model = CelebModel()
trained_model.load_state_dict(torch.load('/content/CelebFaces-Attributes-Prediction/model.pth', map_location=torch.device('cpu')))
trained_model.eval()

# Export the trained model to ONNX
dummy_input = Variable(torch.randn(1, 224, 224, 3))
torch.onnx.export(trained_model, dummy_input, "celeb_model3.onnx")

model = onnx.load('celeb_model3.onnx')
# Import the ONNX model to Tensorflow
tf_rep = prepare(model)
tf_rep.export_graph('celeb_model3.pb')

!zip celeb3.zip -r celeb_model3.pb

INFO:tensorflow:Assets written to: celeb_model3.pb/assets


INFO:tensorflow:Assets written to: celeb_model3.pb/assets


  adding: celeb_model3.pb/ (stored 0%)
  adding: celeb_model3.pb/variables/ (stored 0%)
  adding: celeb_model3.pb/variables/variables.index (deflated 33%)
  adding: celeb_model3.pb/variables/variables.data-00000-of-00001 (deflated 10%)
  adding: celeb_model3.pb/saved_model.pb (deflated 7%)
  adding: celeb_model3.pb/assets/ (stored 0%)
